<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Acuña U.
- Nombre de alumno 2: Martín Guzmán S.


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/sebastianacunau/MDS7202-Laboratorios-y-Proyecto)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
#!pip install -qq xgboost optuna
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import warnings

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

set_config(transform_output="pandas")

#1
random_state = 42
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=random_state, shuffle=True)
validation_data, test_data = train_test_split(temp_data, test_size=1/3, random_state=random_state, shuffle=True)
train_data = train_data.dropna()
validation_data = validation_data.dropna()

#2
def extraer_fecha(df):
    df['day'] = pd.to_datetime(df['date']).dt.day.astype('category') #se convierte a tipo categoria segun lo pedido
    df['month'] = pd.to_datetime(df['date']).dt.month.astype('category') #se convierte a tipo categoria segun lo pedido
    df['year'] = pd.to_datetime(df['date']).dt.year.astype('category') #se convierte a tipo categoria segun lo pedido
    return df

date_transformer = FunctionTransformer(extraer_fecha, validate=False)


df_transformed = date_transformer.fit_transform(train_data)

#3
numerical_columns = df_transformed.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df_transformed.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_columns.remove('quantity')




preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat',  OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough'
)

preprocessor.set_output(transform="pandas")



#4
pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


X_train = train_data.drop(columns=['quantity'])
y_train = train_data['quantity']

pipeline.fit(X_train, y_train)


#5
X_validation = validation_data.drop(columns=['quantity'])
y_validation = validation_data['quantity']

predictions = pipeline.predict(X_validation)
mae = mean_absolute_error(y_validation, predictions)

print(f"MAE Dummy Regressor: {mae:.4f}")

#6
pipeline1 = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(strategy='mean'))
])


X_train = train_data.drop(columns=['quantity'])
y_train = train_data['quantity']

pipeline1.fit(X_train, y_train)

predictions1 = pipeline1.predict(X_validation)
mae1 = mean_absolute_error(y_validation, predictions1)

print(f"MAE XGBOOST: {mae1:.4f}")

MAE Dummy Regressor: 13298.4978
MAE XGBOOST: 2573.8992


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
# Create the pipeline
pipeline_xgb_constrained = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])


# Fit the preprocessor to get feature names
#The change involves applying the fit_transform to the preprocessor directly
# to get the feature names instead of the pipeline slice.
X_train_transformed = pipeline_xgb_constrained['preprocessor'].fit_transform(pipeline_xgb_constrained['date_transformer'].fit_transform(X_train))
feature_names = pipeline_xgb_constrained['preprocessor'].get_feature_names_out()

# Find index of 'price' feature
price_index = list(feature_names).index(next((name for name in feature_names if 'price' in name), None))

# Create monotonic constraints string # This is the updated line
monotonic_constraints = "(" + ",".join(['0'] * price_index + ['-1'] + ['0'] * (len(feature_names) - price_index -1)) + ")"

# Set monotonic constraints in the XGBRegressor
pipeline_xgb_constrained.named_steps['regressor'].monotone_constraints = monotonic_constraints

# Train the pipeline
pipeline_xgb_constrained.fit(X_train, y_train)

# Transform validation data using the same pipeline steps
#validation_data_transformed = pipeline_xgb_constrained[:-1].transform(validation_data) #This line is not used

# Predict and calculate MAE
predictions_constrained = pipeline_xgb_constrained.predict(validation_data)
mae_constrained = mean_absolute_error(validation_data['quantity'], predictions_constrained)
print(f"MAE XGBoost with Monotonic Constraint: {mae_constrained:.4f}")

MAE XGBoost with Monotonic Constraint: 2708.6716


## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
   learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
   n_estimators = trial.suggest_int("n_estimators", 50, 1000)
   max_depth = trial.suggest_int("max_depth", 3, 10)
   max_leaves = trial.suggest_int("max_leaves", 0, 100)
   min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
   reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
   reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
   min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)

   pipeline_optuna = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=random_state  # Setting random seed
        ))
    ])

   pipeline_optuna.set_output(transform="pandas")
    # Train and evaluate
   pipeline_optuna.fit(X_train, y_train)
   predictions_optuna = pipeline_optuna.predict(X_validation)
   mae_optuna = mean_absolute_error(y_validation, predictions_optuna)

    #
   trial.set_user_attr("best_pipeline", pipeline_optuna)

   return mae_optuna


study1 = optuna.create_study(direction='minimize', sampler=TPESampler(seed=random_state))
study1.optimize(objective, timeout=300)

print("Number of trials:", len(study1.trials))

Number of trials: 485


In [ ]:
print("Best trial:")
trial = study1.best_trial
print(f"  Value (MAE): {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Get the best pipeline
best_pipeline = trial.user_attrs["best_pipeline"]

Best trial:
  Value (MAE): 6528.5889
  Params:
    learning_rate: 0.04893953267834104
    n_estimators: 254
    max_depth: 5
    max_leaves: 27
    min_child_weight: 4
    reg_alpha: 0.891587464952773
    reg_lambda: 0.33735306944949706
    min_frequency: 0.8222141714347942


In [ ]:
import pickle
with open('Optuna.pkl', 'wb') as file:
    pickle.dump(best_pipeline, file)

print("Modelo guardado exitosamente.")

Modelo guardado exitosamente.


## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
import optuna
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

optuna.logging.set_verbosity(optuna.logging.WARNING)
random_state = 42

def objective1(trial):
    # Hiperparámetros
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
    min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)

    date_pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer)
    ])
    X_train_trans = date_pipeline.fit_transform(X_train)
    X_val_trans = date_pipeline.transform(X_validation)

    categorical_features_updated = X_train_trans.select_dtypes(include=['object', 'category']).columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_frequency), categorical_features_updated)
        ],
        remainder='passthrough'
    ).set_output(transform="pandas")

    X_train_final = preprocessor.fit_transform(X_train_trans)
    X_val_final = preprocessor.transform(X_val_trans)

    # Modelo con pruning
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=random_state,
        enable_categorical=True,
        # The change involves passing a dictionary with the desired evaluation metric and name.
        # By using {'validation-mae': 'mae'}, we explicitly name the metric and set how it should be calculated.
        eval_metric={'validation-mae': 'mae'},
        callbacks=[XGBoostPruningCallback(trial, "validation-mae")]
    )

    model.fit(
        X_train_final, y_train,
        # Ensure the 'eval_set' is provided and correctly formatted.
        eval_set=[(X_val_final, y_validation)],
        verbose=False,
    )

    preds = model.predict(X_val_final)
    mae = mean_absolute_error(y_validation, preds)
    return mae

study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=random_state))
study.optimize(objective1, timeout=300)

print("Number of trials:", len(study.trials))

study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=random_state))
study.optimize(objective1, timeout=300)

print("Number of trials:", len(study.trials))

[W 2025-05-07 00:20:33,642] Trial 0 failed with parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352} because of the following error: XGBoostError("[00:20:33] /workspace/src/metric/metric.cc:49: Unknown metric function {'validation-mae': 'mae'}\nStack trace:\n  [bt] (0) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x25c1ac) [0x78b4a385c1ac]\n  [bt] (1) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x62660e) [0x78b4a3c2660e]\n  [bt] (2) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5dd0bd) [0x78b4a3bdd0bd]\n  [bt] (3) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5f711d) [0x78b4a3bf711d]\n  [bt] (4) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5f8a2d) [0x78b4a3bf8a2d]\n  [bt] (5) /usr/local/lib/pyt

XGBoostError: [00:20:33] /workspace/src/metric/metric.cc:49: Unknown metric function {'validation-mae': 'mae'}
Stack trace:
  [bt] (0) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x25c1ac) [0x78b4a385c1ac]
  [bt] (1) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x62660e) [0x78b4a3c2660e]
  [bt] (2) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5dd0bd) [0x78b4a3bdd0bd]
  [bt] (3) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5f711d) [0x78b4a3bf711d]
  [bt] (4) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x5f8a2d) [0x78b4a3bf8a2d]
  [bt] (5) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x6f) [0x78b4a3765a1f]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x78b5100f8e2e]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x78b5100f5493]
  [bt] (8) /usr/lib/python3.11/lib-dynload/_ctypes.cpython-311-x86_64-linux-gnu.so(+0x

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
import matplotlib.pyplot as plt

fig = optuna.visualization.plot_optimization_history(study1)
fig.show()

In [ ]:
# 2. Gráfico de coordenadas paralelas
fig1 = optuna.visualization.plot_parallel_coordinate(study1)
fig1.show()

In [ ]:
# 3. Gráfico de importancia de hiperparámetros
fig2 = optuna.visualization.plot_param_importances(study1)
fig2.show()

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error



models = {
    "DummyRegressor": pipeline,
    "XGBoost": pipeline1,
    "XGBoost_Constrained": pipeline_xgb_constrained,
    "XGBoost_Optuna": best_pipeline,
    "XGBoost_Optuna_Pruned": best_pipeline_pruned}

results = []
for model_name, model in models.items():
    if model is not None:
        y_pred_val = model.predict(X_validation)
        mae_val = mean_absolute_error(y_validation, y_pred_val)
        results.append([model_name, mae_val])
    else:
        print(f"Model '{model_name}' not found or not trained. Skipping.")


results_df = pd.DataFrame(results, columns=["Model", "MAE (Validation)"])
print(results_df)

# 2
best_model = results_df.loc[results_df["MAE (Validation)"].idxmin()]
print(f"\nBest model: {best_model['Model']} with MAE {best_model['MAE (Validation)']:.4f}")

# 3
best_model_name = best_model["Model"]

best_model_pipeline = models[best_model_name]

y_pred_test = best_model_pipeline.predict(test_data.drop(columns=['quantity']))
mae_test = mean_absolute_error(test_data['quantity'], y_pred_test)
print(f"\nMAE on test set for {best_model_name}: {mae_test:.4f}")

diff = mae_test - best_model["MAE (Validation)"]

print(f"\nDifference between validation and test MAE: {diff:.4f}")

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>